In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

In [2]:
rdf = pd.read_csv('waMPfile.csv')
codeDF = pd.read_csv('segAPcodeN1.csv')
rdf["nearestAirport"] = codeDF.TargetID
rdf["Distance2Airport"] = codeDF.Distance

In [3]:
def subVal(df, route, val):
    for row in df[df.SR==route].index:
        df.loc[row, 'MP_VALUE'] -= val
    return

In [4]:
subVal(rdf, 10, 88)
subVal(rdf, 41, 38)
subVal(rdf, 205, 26)
subVal(rdf, 530, 16)
subVal(rdf, 397, -11)

In [5]:
traffic = pd.read_excel('../Research Data/alano/Traffic.xlsx')
RouteSeg = traffic.drop_duplicates(subset= ['SR','Begin ARM','End ARM']).loc[:, ('SR','Begin ARM','End ARM')]
RouteSeg.index = range(len(RouteSeg))
print len(RouteSeg)

8909


In [6]:
Routes = list(np.unique(np.array(RouteSeg.loc[:,'SR'])))
mp_file = pd.read_csv('milePostFile.csv')
mp_file['SR'] = mp_file['SR'].convert_objects(convert_numeric=True)
mpRoutes = list(np.unique(np.array(mp_file.loc[:,'SR'])))
routeToRemove = list(set(Routes) - set(mpRoutes))

In [7]:
RouteSeg = RouteSeg[~RouteSeg.SR.isin(routeToRemove)]

In [8]:
print len(RouteSeg)

8688


In [9]:
def ab(val):
    if val < 1:
        return 1
    return int(val)

In [10]:
mp_val = RouteSeg["End ARM"].map(lambda s: ab(s))
RouteSeg["MP_VALUE"] = mp_val

In [11]:
def mergeDFs(df):
    mergedDF = pd.merge(df, rdf, how = 'left',left_on = ['SR', 'MP_VALUE'], right_on = ['SR','MP_VALUE'])
    tdf = mergedDF.drop_duplicates(subset = ['SR','Begin ARM','End ARM'])
    tdf.index = range(len(tdf))
    return tdf

In [12]:
tdf = mergeDFs(RouteSeg)
len(tdf) == len(RouteSeg)

True

In [13]:
criteria = tdf[tdf['UID'].apply(np.isnan)]
missingMP = criteria.loc[:,("SR","MP_VALUE")].drop_duplicates(subset = ["SR", "MP_VALUE"])

In [14]:
def misupto(current,previous):
    res = []
    for i in current: #missing consecutive mp_values
        if i not in previous: #missing single value
            res.append(i)
    return res

In [15]:
mis1 = np.array(missingMP[missingMP.MP_VALUE==1]["SR"])
mis2 = np.array(missingMP[missingMP.MP_VALUE<=2]["SR"])
mis3 = np.array(missingMP[missingMP.MP_VALUE<=3]["SR"])
mis4 = np.array(missingMP[missingMP.MP_VALUE<=4]["SR"])
mis5 = np.array(missingMP[missingMP.MP_VALUE<=5]["SR"])
mis6 = np.array(missingMP[missingMP.MP_VALUE<=6]["SR"])
mis7 = np.array(missingMP[missingMP.MP_VALUE<=7]["SR"])
mis8 = np.array(missingMP[missingMP.MP_VALUE<=8]["SR"])

In [16]:
# print mis1, mis2, mis3, mis4, mis5, mis6, mis7, mis8

In [17]:
m18 = misupto(mis8, mis7)
m17 = misupto(mis7, mis6)
m16 = misupto(mis6, mis5)
m15 = misupto(mis5, mis4)
m14 = misupto(mis4, mis3)
m13 = misupto(mis3, mis2)
m12 = misupto(mis2, mis1)

In [18]:
print m18, m17, m16, m15, m14, m13, m12, mis1

[544, 548] [5, 202] [16, 182] [125, 195] [] [22, 99, 166, 270, 525] [204, 395] [ 17  90 162 181 215 290 291 303 397 410 500 503 510 515 522 529 823 900
 903]


In [19]:
def check(val):
    if np.isnan(val):
        return True
    else:
        return False

In [20]:
def addMPvalue(df):
    for row in range(len(df)):
        if check(df.loc[row, "Distance2Airport"]):
            if df.loc[row, "SR"] == 529:
                df.loc[row, "MP_VALUE"] = 5
            elif df.loc[row, "SR"] == 215:
                df.loc[row, "MP_VALUE"] = 4
            elif df.loc[row, "SR"] == 303:
                df.loc[row, "MP_VALUE"] = 5
            elif df.loc[row, "SR"] == 515:
                df.loc[row, "MP_VALUE"] = 4
            elif ((df.loc[row, "SR"] == 165) & (df.loc[row, "MP_VALUE"] == 10)):
                df.loc[row, "MP_VALUE"] += 1
            elif ((df.loc[row, "SR"] == 397) & (df.loc[row, "MP_VALUE"] == 12)):
                df.loc[row, "MP_VALUE"] += 1
            elif df.loc[row, "SR"] == df.loc[(row-1), "SR"]:
                df.loc[row, "MP_VALUE"] = df.loc[(row-1), "MP_VALUE"]
            elif df.loc[row, "SR"] in mis1:
                df.loc[row, "MP_VALUE"] +=1
            elif df.loc[row, "SR"] in m12:
                df.loc[row, "MP_VALUE"] +=2
            elif df.loc[row, "SR"] in m13:
                df.loc[row, "MP_VALUE"] +=3
            elif df.loc[row, "SR"] in m14:
                df.loc[row, "MP_VALUE"] +=4
            elif df.loc[row, "SR"] in m15:
                df.loc[row, "MP_VALUE"] +=5
            elif df.loc[row, "SR"] in m16:
                df.loc[row, "MP_VALUE"] +=6
            elif df.loc[row, "SR"] in m17:
                df.loc[row, "MP_VALUE"] +=7
            elif df.loc[row, "SR"] in m18:
                df.loc[row, "MP_VALUE"] +=8
            else:
                continue
        else:
            continue

In [21]:
def includeMissing(df):
    for row in range(len(df)):
        if check(df.loc[row, "Distance2Airport"]):
            if df.loc[row, "SR"] == df.loc[(row-1), "SR"]:
                df.loc[row, "nearestAirport"] = df.loc[(row-1), "nearestAirport"]
                df.loc[row, "Distance2Airport"] = df.loc[(row-1), "Distance2Airport"]
#             elif df.loc[row, "SR"] in m18:
#                 df.loc[row, "nearestAirport"] = df.loc[(row+1), "nearestAirport"]
#                 df.loc[row, "Distance2Airport"] = df.loc[(row+1), "Distance2Airport"]
    return

In [22]:
# finalSegWcode.columns = ['SR', 'Begin ARM', 'End ARM', 'NearestAirport', 'Distance2Airport']

In [23]:
# check2 = finalSegWcode[finalSegWcode['Distance2Airport'].apply(np.isnan)]
# print len(check2)

In [24]:
# finalSegWcode.to_csv("SegmentCodeDB.csv")

In [25]:
# print len(mis1), len(m12), len(m13), len(m14), len(m15), len(m16), len(m18)

In [26]:
checkmp = tdf[tdf['UID'].apply(np.isnan)]

In [27]:
# missingMP = checkmp.loc[:,("SR","MP_VALUE")].drop_duplicates(subset = ["SR", "MP_VALUE"])

In [28]:
# addMPvalue(tdf)

In [29]:
# newdf = tdf.loc[:, ('SR','Begin ARM','End ARM', "MP_VALUE")]

In [30]:
def doThis(df):
    mergedDF = pd.merge(df, rdf, how = 'left',left_on = ['SR', 'MP_VALUE'], right_on = ['SR','MP_VALUE'])
    tdf = mergedDF.drop_duplicates(subset = ['SR','Begin ARM','End ARM'])
    tdf.index = range(len(tdf))
    finalSegWcode = tdf.loc[:,('SR','Begin ARM','End ARM', "nearestAirport", "Distance2Airport") ]
    return finalSegWcode

In [31]:
# finalDF = doThis(newdf)
# check3 = finalDF[finalDF['Distance2Airport'].apply(np.isnan)]
# print len(check3)

In [32]:
 def calAndCheck(checkmp, tdf):   
    missingMP = checkmp.loc[:,("SR","MP_VALUE")].drop_duplicates(subset = ["SR", "MP_VALUE"])
    mis1 = np.array(missingMP[missingMP.MP_VALUE==1]["SR"])
    mis2 = np.array(missingMP[missingMP.MP_VALUE<=2]["SR"])
    mis3 = np.array(missingMP[missingMP.MP_VALUE<=3]["SR"])
    mis4 = np.array(missingMP[missingMP.MP_VALUE<=4]["SR"])
    mis5 = np.array(missingMP[missingMP.MP_VALUE<=5]["SR"])
    mis6 = np.array(missingMP[missingMP.MP_VALUE<=6]["SR"])
    mis7 = np.array(missingMP[missingMP.MP_VALUE<=7]["SR"])
    mis8 = np.array(missingMP[missingMP.MP_VALUE<=8]["SR"])
    m18 = misupto(mis8, mis7)
    m17 = misupto(mis7, mis6)
    m16 = misupto(mis6, mis5)
    m15 = misupto(mis5, mis4)
    m14 = misupto(mis4, mis3)
    m13 = misupto(mis3, mis2)
    m12 = misupto(mis2, mis1)
    addMPvalue(tdf)
    newdf = tdf.loc[:, ('SR','Begin ARM','End ARM', "MP_VALUE")]
    TDF = mergeDFs(newdf)
    checkMissing = TDF[TDF['Distance2Airport'].apply(np.isnan)]
    return len(checkMissing), TDF, checkMissing

In [63]:
#have not applied this function yet. I hope it works
def cleanDataFrame(tdf):
    while i < 20:
        n, tdf, checkmp = calAndCheck(checkmp, tdf)
        if n == 0:
            return tdf, checkmp
        i+=1
    return tdf, checkmp

In [35]:
n, tdf2, checkdf = calAndCheck(checkmp, tdf)

In [37]:
n2, tdf3, checkdf = calAndCheck(checkdf, tdf2)

In [39]:
n3, tdf4, checkdf = calAndCheck(checkdf, tdf3)

In [43]:
n4, tdf5, checkdf = calAndCheck(checkdf, tdf4)

In [45]:
n5, tdf6, checkdf = calAndCheck(checkdf, tdf5)

In [51]:
n6, tdf7, checkdf = calAndCheck(checkdf, tdf6)

In [55]:
n7, tdf8, checkdf = calAndCheck(checkdf, tdf7)

In [60]:
n8, tdf9, checkdf = calAndCheck(checkdf, tdf8)

In [61]:
print n8

0


In [64]:
np.unique(np.array(checkdf.loc[:, "SR"]))

array([], dtype=int64)

In [ ]:
# checkmp[checkmp.SR==303]

In [ ]:
# RouteSeg[RouteSeg.SR==397]

In [65]:
print len(tdf9) == len(RouteSeg)

True


In [ ]:
reqdf = tdf9.loc[:, ('SR', 'Begin ARM', 'End ARM', 'nearestAirport', 'Distance2Airport')]
reqdf.columns = ['SR', 'Begin ARM', 'End ARM', 'NearestAirport', 'Distance2Airport']

In [66]:
tdf9.to_csv("SegmentCodeDB.csv")